In [1]:
import matplotlib.pyplot as plt

import numpy as np
import scipy.io as sio
from scipy.stats import ortho_group
from sklearn.svm import SVC 
import math 
import random
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.linear_model import orthogonal_mp

import time

In [2]:
cd Desktop/LANL/AFRL-DISC Data/Blackbox brief for LANL

C:\Users\khenk\Desktop\LANL\AFRL-DISC Data\Blackbox brief for LANL


In [3]:
sPCA= pd.read_csv('sPCA_32.csv', sep=',', encoding = 'utf8')
data = sPCA.values
#Load sPCA Dictionary from found sPCA components
mat = sio.loadmat('sPCA_Phi.mat')
sPCA_Phi = mat['sPCA_Phi']

mat = sio.loadmat('fMnist_ytrainlabels.mat')
y_train = np.squeeze(np.asarray(mat['y_train'].T))
mat = sio.loadmat('fMnist_ytestlabels.mat')
y_test = np.squeeze(np.asarray(mat['y_test'].T))


In [4]:
Reconstructed_Data = np.dot(data,sPCA_Phi)
Reconstructed_Data = Reconstructed_Data.reshape(len(Reconstructed_Data),28,28)

In [5]:
#Cutting up image
def get_cell(img, x, y):
    return img[x:x+span,y:y+span].tolist()

In [8]:
#Patch up data
full_set = []
row_size = len(Reconstructed_Data[1])
col_size = len(Reconstructed_Data[2])

span = 7



row_size_list = 4
col_size_list = 4

for i in range(len(Reconstructed_Data)):
    cropped_image = Reconstructed_Data[i]
    
    #normailze image with mean 0 and unit variance
    cropped_image = (cropped_image - np.mean(cropped_image)) / (np.std(cropped_image))
   
    submatrix_list = [get_cell(cropped_image,x,y) for x in range(0,row_size,span) for y in range(0,col_size,span)]
    
    
    d2_list = [submatrix_list[i:i+row_size_list] for i in range(0,len(submatrix_list),row_size_list)]
    #Validation of chopping
    X = np.zeros((row_size,col_size))
    for i in range(row_size_list):
        for j in range(col_size_list):
            for k in range(span):
                for l in range(span):
                    X[i*span+k,j*span+l] = d2_list[i][j][k][l]
    full_set.append(d2_list)
    

Data_Patches = []
for i in range(len(Reconstructed_Data)):
    for j in range(row_size_list):
        for k in range(col_size_list):
            Sample_image_patch = full_set[i][j][k]
            Data_Patches.append(Sample_image_patch)

In [9]:
Data_Patches = np.asarray(Data_Patches)
Data_Patches = Data_Patches.reshape(len(Data_Patches),len(Data_Patches[1])*len(Data_Patches[2]))

In [10]:
#normalize data
for i in range(len(Data_Patches)):
    if LA.norm(Data_Patches[i])!=0:
        Data_Patches[i]=Data_Patches[i]/LA.norm(Data_Patches[i])
# Data_Patches= np.asarray(Data_Patches)
Data_Patches.shape

(1120000L, 49L)

In [11]:
#Creating Randomly imprinted Phi Matrix with number of features = # Neurons(qubits)
Neurons = 64
RandomTrainingSample_Index = random.sample(range(len(Data_Patches)),Neurons)
Rand_Features = Data_Patches[RandomTrainingSample_Index,:]
Rand_Features.shape

(64L, 49L)

In [12]:
#Get Features for data phi and Normalize
Phi = Rand_Features.T
for i in range(Neurons):
    Phi[:,i]= Phi[:,i]/(LA.norm(Phi[:,i]))

In [13]:
def sparse_loss(I,Phi,sparse_representation_coefficients,lam):
    Energy = .5*LA.norm(I-np.dot(Phi,sparse_representation_coefficients))**2 +lam*np.count_nonzero(sparse_representation_coefficients)
    return Energy

def reconstruction_loss(I,Phi,sparse_representation_coefficients):
    Loss = LA.norm(I-np.dot(Phi,sparse_representation_coefficients))**2
    return Loss

In [14]:
#SGD Loop to update Dictionary
Data_Patches = Data_Patches
mini_batch_size = 512
tol = 10**(-32)
max_iterations = 100000
eta =.0001 #Learning Rate (Lower is better)
lam = .35 #found through orthogonal feature reconstructions above
lam_vector = np.full((len(Phi.T), 1), lam, dtype=np.float)
count = 0
Previous_Gradient = 0
momentum_term = .9



Total_Energies = []
Total_Recon_Loss = []
Non_Zero_Coefficients =[] #Helps see how sparse reconstructions are while learning
eta_initial =.01
eta_final = .000001#Learning Rate (Lower is better)
eta = eta_initial
for iteration in range(max_iterations):
    
        
    #Random Draw from Data
    Indexing = random.sample(range(len(Data_Patches)), mini_batch_size)
    
    Neuron_Activation =[]
    Gradients =[]
    Unembedded_Solutions_Training= []
    t1 = time.time()
    New_Solutions = []
    Temp_Data_Patches=[]
    Q = .5*np.dot(np.squeeze(Phi.T),np.squeeze(Phi))
    


    image_number = 0
    Sparse_Rep=[]
    Loss = []
    Sparsity_Loss = []
    Gradients = []
    for sample in Indexing:
        temp = orthogonal_mp(Phi,Data_Patches[sample])
        Sparsity_Loss.append((sparse_loss(Data_Patches[sample],Phi,temp.T,lam)))
        Sparse_Rep.append(temp)
        Loss.append(LA.norm(Data_Patches[sample]-np.dot(Phi,orthogonal_mp(Phi,Data_Patches[sample]))))
        error = np.subtract(Data_Patches[sample].T,np.squeeze(np.dot(Phi,temp.T)))
        Delta_Phi =np.outer(error,temp)
        Delta_Phi = np.asarray(Delta_Phi)
        Gradients.append(Delta_Phi)
        
    Gradient = sum(np.asarray(Gradients))/mini_batch_size
    #print(Gradient.shape)
    #Phi_New = Phi - (eta/mini_batch_size)*np.squeeze(Delta_Phi).T + momentum_term*Previous_Gradient
    Phi_New = Phi - (eta)*Gradient + momentum_term*Previous_Gradient #non division by mini batch size
    Previous_Gradient = Gradient
    
    for i in range(Neurons):
        Phi_New[:,i]= Phi_New[:,i]/(math.sqrt(np.dot(Phi_New[:,i].T,Phi_New[:,i])))
    Previous_Gradient = Gradient 

    Loss = sum(np.asarray(Loss))/len(Indexing)
    Sparsity_Loss = sum(np.asarray(Sparsity_Loss))/len(Indexing)
    Total_Energies.append(Sparsity_Loss)
    Total_Recon_Loss.append(Loss)

#
    
    if count%10==0:
        print(count)
        print(Loss,Sparsity_Loss,np.count_nonzero(Sparse_Rep))
        print(LA.norm(Phi_New-Phi))
    #print(plt.plot(range(iteration+1), Total_Energies ))
    #print('Energies,Epoc :', Total_Energies,count)
    elapsed1 = time.time() - t1
    #print(elapsed1)
    count += 1

#     sio.savemat('Phi_New_Fashion_MiniBatch1000_sPCA_AFRL_32_Lambda_'+str(lam)+'_OMP.mat', {'Phi_New':Phi_New})
#     sio.savemat('TotalEnergies_Fashion_MiniBatch1000_sPCA_AFRL_32_Lambda_'+str(lam)+'_OMP.mat', {'Total_Energies':Total_Energies})
    if LA.norm(Phi_New-Phi) > tol:
        #print(LA.norm(Phi_New-Phi))
        Phi = Phi_New
        eta = eta_initial*((eta_final/float(eta_initial))**(iteration/float(max_iterations))) #exponentially decaying learing rate


    else:
        print('converged')
        
        break


0
(0.17665513314802156, 2.118871576846369, 3072)
0.00015013387649002433
10
(0.1781932672192071, 2.119110030965031, 3072)
0.01268547278841093
20
(0.17320484219549662, 2.1184730973796375, 3072)
0.012203322889231824
30
(0.17132769494502123, 2.1181698523200154, 3072)
0.010521190494426068
40
(0.1655067652890869, 2.1166411376156513, 3072)
0.010751956526318038
50
(0.16453952692804466, 2.1164380094599635, 3072)
0.010073019169005264
60
(0.16815628872711208, 2.1173747992439873, 3072)
0.009687315893373011
70
(0.16261316938324993, 2.1161178079082523, 3072)
0.009724845467980801
80
(0.1560215111132675, 2.1146198309570767, 3072)
0.009635895658761592
90
(0.1673545478855996, 2.116660921740773, 3072)
0.008936279068310557
100
(0.16171540829952621, 2.1152913962205235, 3072)
0.008759085986041935
110
(0.1647535334740342, 2.116625722544411, 3072)
0.0083867345850698
120
(0.1616146748450357, 2.115637982981431, 3072)
0.008340465888514798
130
(0.1627778106776601, 2.116155206861767, 3072)
0.007512617042010185
140

1140
(0.1498497547911642, 2.113323226730909, 3072)
0.006894649474913816
1150
(0.14652969025552312, 2.112657318361886, 3072)
0.007365240614149048
1160
(0.14749418858466232, 2.1131322655965747, 3072)
0.0072614414486951735
1170
(0.15008432339798294, 2.1132095164060227, 3072)
0.006943516118197643
1180
(0.1509056577580504, 2.113720315338605, 3072)
0.007612405692538089
1190
(0.14907779888501424, 2.112991334720141, 3072)
0.006818057298293839
1200
(0.1463161273722049, 2.1129691167314153, 3072)
0.007244445584234932
1210
(0.14429417919231236, 2.1123063375481332, 3072)
0.007030027559663528
1220
(0.14620762356321215, 2.1130062133989345, 3072)
0.0073069066277726205
1230
(0.1441190527381024, 2.112661769753392, 3072)
0.00758121512839299
1240
(0.145528665601993, 2.112529853216041, 3072)
0.006852609795679608
1250
(0.14428020081705892, 2.11228533862334, 3072)
0.006861423969064027
1260
(0.1427309010596827, 2.1124603546233236, 3072)
0.0069012251051236975
1270
(0.14253651601553918, 2.1121522295595945, 3072

KeyboardInterrupt: 

In [17]:
np.save('AFRL_Recon_Patches', Data_Patches)
np.save('Phi_Recon_OMP', Phi_New)